In [1]:
import logging
import sys
import os
from dotenv import load_dotenv
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()

import llama_index
llama_index.__version__

'0.9.42.post1'

In [2]:
from llama_index import SimpleDirectoryReader

# filename_as_id = True로 하면 node_id가 file_name_part_{num} 으로 설정됨
llama2_doc = SimpleDirectoryReader(input_files=["./assets/llama2.pdf"], filename_as_id=True).load_data()

In [3]:
print(len(llama2_doc)) # 77 페이지

77


In [4]:
llama2_doc[0].node_id # node id가 경로로 설정됨

'assets/llama2.pdf_part_0'

In [5]:
from llama_index.extractors import (
    QuestionsAnsweredExtractor,
    TitleExtractor
)

from llama_index.node_parser import SimpleNodeParser

# documents를 chunking
node_parser = SimpleNodeParser(separator=" ", chunk_size=512, chunk_overlap=20, include_metadata=True)
llama2_nodes = node_parser.get_nodes_from_documents(llama2_doc)

In [6]:
llama2_nodes[0] # ref_doc_id가 청킹되기 전 documents의 id로 설정됨

TextNode(id_='6e3f7640-4647-4da7-ba12-26f99f4dc5e6', embedding=None, metadata={'page_label': '1', 'file_name': 'llama2.pdf', 'file_path': 'assets/llama2.pdf', 'file_type': 'application/pdf', 'file_size': 13661300, 'creation_date': '2024-02-11', 'last_modified_date': '2023-12-16', 'last_accessed_date': '2024-02-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='assets/llama2.pdf_part_0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'llama2.pdf', 'file_path': 'assets/llama2.pdf', 'file_type': 'application/pdf', 'file_size': 13661300, 'creation_date': '2024-02-11', 'last_modified_date': '2023-12-16', 'last_accessed_date': '2024-02-11'}, hash='9479873a161d4f1f6a832af9688cbd

In [9]:
llama2_nodes[0].ref_doc_id

'assets/llama2.pdf_part_0'

In [10]:
llama2_nodes[10].ref_doc_id

'assets/llama2.pdf_part_4'

In [11]:
llama2_nodes[100].ref_doc_id

'assets/llama2.pdf_part_39'

In [17]:
llama2_nodes.__len__()

196

In [18]:
from llama_index import ServiceContext, VectorStoreIndex

# service_context = ServiceContext.from_defaults(node_parser=node_parser)
index = VectorStoreIndex.from_documents(llama2_doc)#, service_context=service_context)
index.storage_context.persist(persist_dir='./storage/cache/papers/llama2_node_id/')

In [19]:
# ref_docs는 유지보수를 위해 필요할 것 같음
# delete_from_docstore = True로 설정하면 docstore에서 새로운게 있으면 True로 반환됨. False로 반환되면 새로운 docs가 없는 것
refreshed_docs = index.refresh_ref_docs(documents=llama2_doc, update_kwargs={"delete_kwargs":{"delete_from_docstore":True}})
print(refreshed_docs)
print("Num of inserted/refreshed documents : ", sum(refreshed_docs))

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Num of inserted/refreshed documents :  0


In [20]:
# 경로에 다른 파일을 추가함
from llama_index import SimpleDirectoryReader
from llama_index import ServiceContext, VectorStoreIndex

docs = SimpleDirectoryReader("./assets", filename_as_id=True).load_data()
refreshed_docs = index.refresh_ref_docs(documents=docs, update_kwargs={"delete_kwargs":{"delete_from_docstore":True}})
print(refreshed_docs)
print("Num of inserted/refreshed documents : ", sum(refreshed_docs))

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Num of inserted/refreshed documents :  26


In [21]:
# 위에서 True로 나왔으니 인덱스를 다시 업데이트 해야 함.
index = VectorStoreIndex.from_documents(docs)
index.storage_context.persist(persist_dir='./storage/cache/papers/llama2_node_id/')

In [22]:
docs = SimpleDirectoryReader("./assets", filename_as_id=True).load_data()
refreshed_docs = index.refresh_ref_docs(documents=docs, update_kwargs={"delete_kwargs":{"delete_from_docstore":True}})
print(refreshed_docs)
print("Num of inserted/refreshed documents : ", sum(refreshed_docs))

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Num of inserted/refreshed documents :  0
